In [1]:
import os
import pickle
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from scipy import ndimage, misc
from sklearn import datasets
from sklearn.model_selection import train_test_split
%matplotlib inline
# 乱数シードを指定
np.random.seed(seed=0)

In [2]:
#CIFAR-10のデータセットのインポート
from keras.datasets import cifar10
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
#CIFAR-10の正規化
from keras.utils import to_categorical 
# 特徴量の正規化
X_train = X_train/255.
X_test = X_test/255.
# クラスラベルの1-hotベクトル化
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

In [3]:
#CIFAR-100のデータセットのインポート
from keras.datasets import cifar100
(X_train2, Y_train2), (X_test2, Y_test2) = cifar100.load_data()
#CIFAR-100の正規化
from keras.utils import to_categorical
# 特徴量の正規化
X_train2 = X_train2/255.
X_test2 = X_test2/255.
 
# クラスラベルの1-hotベクトル化
Y_train2 = to_categorical(Y_train2, 100)
Y_test2 = to_categorical(Y_test2, 100)

#score2 = model.evaluate(X_test2, Y_test0)

In [ ]:
# CNNの構築
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
import numpy as np
 
model = Sequential()
 
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))
 
# コンパイル
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
 
#訓練
history = model.fit(X_train, Y_train, epochs=20)

# モデルの保存
model.save('./CIFAR-10.h5')

In [4]:
from keras.models import load_model
# 保存したモデル構造の読み込み
model=load_model('./CIFAR-10.h5')
 #評価 & 評価結果出力
print(model.evaluate(X_test, Y_test))

313/313 [==============================] - 7s 23ms/step - loss: 0.7757 - accuracy: 0.7311
[0.775740921497345, 0.7311000227928162]


In [9]:
import numpy as np
# cifar-10 : (60000, 32, 32), (60000)
# cifar-100 : (60000, 32, 32), (60000)
# X_test : (10000, 32, 32)
# Y_test : (10000, 10)
# Y_test01_10_10000 : [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]  を10000個並べたもの
Y_test01_10_10000 = np.full((10000, 10), 0.1)
print(model.evaluate(X_test, Y_test01_10_10000))
print(model.evaluate(X_test2, Y_test01_10_10000))

313/313 [==============================] - 10s 32ms/step - loss: 6.4291 - accuracy: 0.0929
[6.429050922393799, 0.09290000051259995]
313/313 [==============================] - 8s 25ms/step - loss: 4.5705 - accuracy: 0.0832
[4.570457935333252, 0.08320000022649765]


In [10]:
# Y_test01_10ではすべてが均等な確率なので、モデルによる予測の確信が最も低いという事ができる。
# これとのlossが少ないものほど予測の確信度が低い、
# つまり、cifar-10の中での10クラス分類がうまくいかないと言える。これはcifar-100のデータである可能性が高い。
# したがって、lossの値を適切に選べば精度良くcifar-100を識別する事ができる可能性がある。
# 今回cifar-10, cifar-100のlossの平均はそれぞれ6.43, 4.57と差が小さいが、この間に境界を設定すれば分類できる可能性が高い。

In [11]:
# y_test01_10 : 200個のデータセットに対して、[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] を並べたもの。
y_test01_10 = np.full((200, 10), 0.1)
# x_judge : (200, 3072) : 予測精度確認用の画像の入力データセット
x_judge = np.zeros((200, 32, 32, 3))
# cifar-10, cifar-100からそれぞれ100ずつデータを取ってきている。
x_judge[0:100,:,:,:] = X_test[0:100,:,:,:]
x_judge[100:,:,:,:] = X_test2[100:200,:,:,:]
# y_judge : (20000,) : 今回前半の100個がcifar-10(0とする)、残りの100個がcifar-100(1とする)としている。
y_judge = np.full(200, 0)
y_judge[100:] = np.full(100, 1)

In [32]:
# lossがkより少ないものを1(cifar-10)、そうでないものを0(cifar-100)と予測している。
j = np.zeros(200)
k = 5.5
for i in range(200) :
    loss = model.evaluate(x_judge[i:i+1,:,:,:], y_test01_10[i:i+1,:], verbose=0)
    if loss[0]<k:
            j[i] =1

In [33]:
cnt_1 = 0.0
cnt_2 = 0.0
for i in range(100) :
    #print(j[i])
    #print(y_judge[i])
    if(j[i] == y_judge[i]):
        cnt_1+=1.0
for i in range(100) :
    #print(j[i+100])
    #print(y_judge[i+100])
    if(j[i+100] == y_judge[i+100]):
        cnt_2+=1.0
ans_1 = cnt_1 / 100
ans_2 = cnt_2 / 100
ans = (cnt_1+cnt_2) / 200
print("cifar-10 accracy :",ans_1)
print("cifar-100 accracy :",ans_2)
print("accuracy :",ans)

cifar-10 accracy : 0.56
cifar-100 accracy : 0.84
accuracy : 0.7


In [25]:
# k = 6.0, cifar-10 accracy : 0.48, cifar-100 accracy : 0.9, accuracy : 0.69
# k = 5.5, cifar-10 accracy : 0.56, cifar-100 accracy : 0.84, accuracy : 070
# k = 5.0, cifar-10 accracy : 0.64, cifar-100 accracy : 0.74, accuracy : 0.69
# k = 4.5, cifar-10 accracy : 0.75, cifar-100 accracy : 0.63, accuracy : 0.69
# k = 4.0, cifar-10 accracy : 0.85, cifar-100 accracy : 0.43, accuracy : 0.43
# k = 5.5が最適と言える。

In [34]:
# 逆に[0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0]のように0を9個、1.0を1個並べたものすべてとのlossを比べて、
# これらとのlossが1つでもkより低くなるものをcifar-10だと判定し、余ったものをcifar-100と判定するという手法も考えられる。

In [46]:
# y_test10_1 : (200, 10) : [0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0]のように0を9個、1.0を1個並べたもの
y_test10_1= np.full((200, 10), 0.0)
# 初めにすべてを1(cifar-100)と設定する。
j = np.full(200, 1)
# 10個それぞれの要素の一つ一つが1.0になる場合にlossがk以下のものを0つまり(cifar-10)と適宜変えていく。
for i_x in range(10):
            y_test10_1[:,:]=0.0
            y_test10_1[:,i_x] = 1.0
            for i in range(200) :
                loss = model.evaluate(x_judge[i:i+1,:,:,:], y_test10_1[i:i+1,:], verbose=0)
                if loss[0]<0.6 :
                    j[i] = 0

In [47]:
cnt_1 = 0.0
cnt_2 = 0.0
for i in range(100) :
    #print(j[i])
    #print(y_judge[i])
    if(j[i] == y_judge[i]):
        cnt_1+=1.0
for i in range(100) :
    #print(j[i+100])
    #print(y_judge[i+100])
    if(j[i+100] == y_judge[i+100]):
        cnt_2+=1.0
ans_1 = cnt_1 / 100
ans_2 = cnt_2 / 100
ans = (cnt_1+cnt_2) / 200
print("cifar-10 accracy :",ans_1)
print("cifar-100 accracy :",ans_2)
print("accuracy :",ans)

cifar-10 accracy : 0.76
cifar-100 accracy : 0.63
accuracy : 0.695


In [48]:
# k = 0.3 cifar-10 accracy : 0.53, cifar-100 accracy : 0.87, accuracy : 0.70
# k = 0.5, cifar-10 accracy : 0.70, cifar-100 accracy : 0.74, accuracy : 072
# k = 0.6, cifar-10 accracy : 0.76, cifar-100 accracy : 0.63, accuracy : 0.695
# k = 0.7, cifar-10 accracy : 0.79, cifar-100 accracy : 0.51, accuracy : 0.65
# k = 0.5が最適と言える。

In [49]:
# このモデルの精度はy_test10_1((200, 10) : [0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0]のように0を9個、1.0を1個並べたもの)とのlossをk=0.5を境界に
# 分けたものの72%が最高。
# このモデルでは精度に限界がある。さらに精度の良いモデルを作る。

In [ ]:
# CNNの構築
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
 
model = Sequential()
 
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(GlobalAveragePooling2D())

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

# コンパイル
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
 
#訓練
history = model.fit(X_train, Y_train, epochs=50)

# モデルの保存
model.save('./CIFAR-10_4.h5')

# 工夫
# 1. 層を厚くした。4層しかなかったConv2D層を10個に増やし、適宜正規化用のDropout()やBatchNormalization(),
# GlobalAveragePooling2D()などを加えた。
# 2. エポック数を増やした。

In [51]:
# 保存したモデル構造の読み込み
model=load_model('./CIFAR-10_4.h5')
#評価 & 評価結果出力
print(model.evaluate(X_test, Y_test))

313/313 [==============================] - 26s 84ms/step - loss: 0.5608 - accuracy: 0.8437
[0.5608178973197937, 0.8436999917030334]


In [52]:
# 精度が84%に(11%向上)

In [53]:
import numpy as np
# cifar-10 : (60000, 32, 32), (60000)
# cifar-100 : (60000, 32, 32), (60000)
# X_test : (10000, 32, 32)
# Y_test : (10000, 10)
# Y_test01_10_10000 : [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]  を10000個並べたもの
Y_test01_10_10000 = np.full((10000, 10), 0.1)
print(model.evaluate(X_test, Y_test01_10_10000))
print(model.evaluate(X_test2, Y_test01_10_10000))

313/313 [==============================] - 22s 70ms/step - loss: 11.6481 - accuracy: 0.1011
[11.648058891296387, 0.10109999775886536]
313/313 [==============================] - 25s 81ms/step - loss: 6.8170 - accuracy: 0.0992
[6.817044734954834, 0.09920000284910202]


In [54]:
# 今回cifar-10, cifar-100のlossの平均はそれぞれ11.65, 6.82と差が大きくなった。この間に境界を設定すれば分類できる可能性が高い。

In [66]:
# lossがkより少ないものを1(cifar-10)、そうでないものを0(cifar-100)と予測している。
j = np.zeros(200)
k = 8.0
for i in range(200) :
    loss = model.evaluate(x_judge[i:i+1,:,:,:], y_test01_10[i:i+1,:], verbose=0)
    if loss[0]<k:
            j[i] =1

In [67]:
cnt_1 = 0.0
cnt_2 = 0.0
for i in range(100) :
    #print(j[i])
    #print(y_judge[i])
    if(j[i] == y_judge[i]):
        cnt_1+=1.0
for i in range(100) :
    #print(j[i+100])
    #print(y_judge[i+100])
    if(j[i+100] == y_judge[i+100]):
        cnt_2+=1.0
ans_1 = cnt_1 / 100
ans_2 = cnt_2 / 100
ans = (cnt_1+cnt_2) / 200
print("cifar-10 accracy :",ans_1)
print("cifar-100 accracy :",ans_2)
print("accuracy :",ans)

cifar-10 accracy : 0.67
cifar-100 accracy : 0.8
accuracy : 0.735


In [68]:
# k = 5.5, cifar-10 accracy : 0.93, cifar-100 accracy : 0.35, accuracy : 0.64
# k = 7.0, cifar-10 accracy : 0.84 cifar-100 accracy : 0.68, accuracy : 0.76
# k = 7.5, cifar-10 accracy : 0.76 cifar-100 accracy : 0.77, accuracy : 0.765
# k = 8.0, cifar-10 accracy : 0.75 cifar-100 accracy : 0.79, accuracy : 0.77
# k = 8.5, cifar-10 accracy : 0.67 cifar-100 accracy : 0.80, accuracy : 0.735
# k = 9.0, cifar-10 accracy : 0.63cifar-100 accracy : 0.81 accuracy : 0.72
#k = 8.0が最適

In [86]:
# y_test10_1 : (200, 10) : [0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0]のように0を9個、1.0を1個並べたもの
y_test10_1= np.full((200, 10), 0.0)
# 初めにすべてを1(cifar-100)と設定する。
j = np.full(200, 1)
k = 0.02
# 10個それぞれの要素の一つ一つが1.0になる場合にlossがk以下のものを0つまり(cifar-10)と適宜変えていく。
for i_x in range(10):
            y_test10_1[:,:]=0.0
            y_test10_1[:,i_x] = 1.0
            for i in range(200) :
                loss = model.evaluate(x_judge[i:i+1,:,:,:], y_test10_1[i:i+1,:], verbose=0)
                if loss[0]<k :
                    j[i] = 0

In [87]:
cnt_1 = 0.0
cnt_2 = 0.0
for i in range(100) :
    #print(j[i])
    #print(y_judge[i])
    if(j[i] == y_judge[i]):
        cnt_1+=1.0
for i in range(100) :
    #print(j[i+100])
    #print(y_judge[i+100])
    if(j[i+100] == y_judge[i+100]):
        cnt_2+=1.0
ans_1 = cnt_1 / 100
ans_2 = cnt_2 / 100
ans = (cnt_1+cnt_2) / 200
print("cifar-10 accracy :",ans_1)
print("cifar-100 accracy :",ans_2)
print("accuracy :",ans)

cifar-10 accracy : 0.63
cifar-100 accracy : 0.87
accuracy : 0.75


In [81]:
# k = 0.01, cifar-10 accracy : 0.56, cifar-100 accracy : 0.91, accuracy : 0.735
# k = 0.02, cifar-10 accracy : 0.63, cifar-100 accracy : 0.87, accuracy : 0.75
# k = 0.03, cifar-10 accracy : 0.69, cifar-100 accracy : 0.86, accuracy : 0.775
# k = 0.05, cifar-10 accracy : 0.71, cifar-100 accracy : 0.77, accuracy : 0.74
# k = 0.1, cifar-10 accracy : 0.76, cifar-100 accracy : 0.62, accuracy : 0.69
# k = 0.2, cifar-10 accracy : 0.85, cifar-100 accracy : 0.5, accuracy : 0.675
# k = 0.5, cifar-10 accracy : 0.95, cifar-100 accracy : 0.26, accuracy : 0.605
# k = 0.6, cifar-10 accracy : 0.96, cifar-100 accracy : 0.24, accuracy : 0.60
# k = 0.03が最適と言える。

In [82]:
# このモデルの精度はy_test01_10([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] を200個並べたもの)とのlossをk=8.0を境界に
# 分けたものの77%または、y_test10_1((200, 10) : [0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0]のように0を9個、1.0を1個並べたものを200個並べたもの)
# とのlossをk=0.03を境界に分けたものの77.5%が最高。(5.5%の精度向上が見られた。)
# この2つをcifar-10, 100の全てのテストケースに試して精度を確かめる。

In [90]:
# y_test01_10_20000 : 20000個のデータセットに対して、[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] を並べたもの。
y_test01_10_20000 = np.full((20000, 10), 0.1)
# x_judge2 : (20000, 3072) : 予測精度確認用の画像の入力データセット
x_judge2 = np.zeros((20000, 32, 32, 3))
# cifar-10, cifar-100からそれぞれ100ずつデータを取ってきている。
x_judge2[0:10000,:,:,:] = X_test
x_judge2[10000:,:,:,:] = X_test2
# y_judge2 : (20000,) : 今回前半の100個がcifar-10(0とする)、残りの100個がcifar-100(1とする)としている。
y_judge2 = np.full(20000, 0)
y_judge2[10000:] = np.full(10000, 1)

In [ ]:
# lossがkより少ないものを1(cifar-10)、そうでないものを0(cifar-100)と予測している。
j = np.zeros(20000)
k = 9.0
for i in range(20000) :
    loss = model.evaluate(x_judge2[i:i+1,:,:,:], y_test01_10_20000[i:i+1,:], verbose=0)
    if loss[0]<k:
            j[i] =1

In [ ]:
cnt_1 = 0.0
cnt_2 = 0.0
for i in range(10000) :
    #print(j[i])
    #print(y_judge[i])
    if(j[i] == y_judge2[i]):
        cnt_1+=1.0
for i in range(10000) :
    #print(j[i+100])
    #print(y_judge[i+100])
    if(j[i+10000] == y_judge2[i+10000]):
        cnt_2+=1.0
ans_1 = cnt_1 / 10000
ans_2 = cnt_2 / 10000
ans = (cnt_1+cnt_2) / 20000
print("cifar-10 accracy :",ans_1)
print("cifar-100 accracy :",ans_2)
print("accuracy :",ans)

In [ ]:
# k = 7.5, cifar-10 : 0.7759, cifar-100 : 0.69, accuracy : 0.73295
# k = 8.0, cifar-10 : 0.7336, cifar-100 : 0.7555, accuracy : 0.74455
# k = 8.5, cifar-10 : 0.6881, cifar-100 : 0.8105, accuracy : 0.7493

In [99]:
# y_test10_1_20000 : (20000, 10) ([0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0]のように0を9個、1.0を1個並べたものを20000個並べたもの)
y_test10_1_20000 = np.full((20000, 10), 0.0)
# 初めにすべてを1(cifar-100)と設定する。
j = np.full(20000, 1)
# 10個それぞれの要素の一つ一つが1.0になる場合にlossがk以下のものを0つまり(cifar-10)と適宜変えていく。
for i_x in range(10):
            y_test10_1_20000[:,:]=0.0
            y_test10_1_20000[:,i_x] = 1.0
            for i in range(20000) :
                loss = model.evaluate(x_judge2[i:i+1,:,:,:], y_test10_1_20000[i:i+1,:], verbose=0)
                if loss[0]<0.03 :
                    j[i] = 0

In [101]:
cnt_1 = 0.0
cnt_2 = 0.0
for i in range(10000) :
    #print(j[i])
    #print(y_judge[i])
    if(j[i] == y_judge2[i]):
        cnt_1+=1.0
for i in range(10000) :
    #print(j[i+10000])
    #print(y_judge[i+10000])
    if(j[i+10000] == y_judge2[i+10000]):
        cnt_2+=1.0
ans_1 = cnt_1 / 10000
ans_2 = cnt_2 / 10000
ans = (cnt_1+cnt_2) / 20000
print("cifar-10 accracy :",ans_1)
print("cifar-100 accracy :",ans_2)
print("accuracy :",ans)

cifar-10 accracy : 0.676
cifar-100 accracy : 0.7764
accuracy : 0.7262


In [104]:
# k = 0.03, cifar-10 : 0.676, cifar-100 : 0.7764, accuracy : 0.7262
# 時間がかかり過ぎる上に見込みがないので割愛

In [ ]:
# このモデルの精度はy_test01_10_20000 ([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] を20000個並べたもの)とのlossをk=8.5を境界に
# 分けて、大きいものをcifar-100、小さいものをcifar-10とすることによって74.9%の精度を達成する事ができた。